## Visual Document Classifier Lilt

In [0]:
import os
import matplotlib.pyplot as plt
from pyspark.ml import Pipeline, PipelineModel

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

import sparkocr
from sparkocr.transformers import *
from sparkocr.utils import *
from sparkocr.enums import *
from sparkocr.databricks import display_images

sparkocr.info()

19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
Spark version: 3.1.2
Spark NLP version: 5.1.0
Spark NLP for Healthcare version: 5.1.0
Spark OCR version: 5.0.0

19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0

In [0]:
import pkg_resources

from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0
19:33:05, INFO Received command c on object id p0

In [0]:
%fs ls /FileStore/examples/ocr/images/tiff/00556614_00556648.tif

path,name,size
dbfs:/FileStore/examples/ocr/images/tiff/00556614_00556648.tif,00556614_00556648.tif,375950


## Load test images

In [0]:
file_path='dbfs:/FileStore/examples/ocr/images/tiff/00556614_00556648.tif'
doc_df = spark.read.format("binaryFile").load(file_path)
display(doc_df)

path modificationTime length content dbfs:/FileStore/examples/ocr/images/tiff/00556614_00556648.tif 2023-09-13T19:45:06.000+0000 375950 List(SUkqALi5BQCB/4H/yf8D5//56oH/gf+B/63/5f8C6vz9gf+B/4H/gf+B/5D/7v8B/P75/wOtdPDSgf+B/4H/gf+B/6r/Adz06//u/w9zUnJoW02+9unGx491iv/8/f8A/YH/gf+N/wD+gf+B/6b/ATWq6//w/xH7iwIAHjwEaNU= (truncated), iVBORw0KGgoAAAANSUhEUgAAAE4AAABkCAAAAAAN9lmRAAASgElEQVR42u1Z6W9l51mf/wapSIgPKEJCqqBqREEpi0oRoaQUEZWIliEVaQuFBoLaKglhCLSZSTLJrMlsscceezz2eBnv+9jX9r2+9/ru+3Lu2dd3O+/hec71MkvaQj/xgdf29bn2Ob/32Z/f895TkX3TkmH0s5eM/ifLjKJTEXtYUxQRwlckZfyk7K9Qxijh4fsw/GlbcXww4EGOh6ciOb20rKh6nQl4QjAaSSZ8l4Uh42HgMIf6jsN5yHzPFFxw3EUCRrxdGEpGXcqIWdjaW9pM+aeiKD12bzqjWEFAXM7gq/8kcQPf8qmutHQjkxobztSbFdVxVIsL4hDKhMsI58TsVPZVL1/afDB5dpyAslHtwvxyqeO5fhjA9oSH8BIw6ruUinZrc311eXJze+RqYmuh2TMYCYjDhOCMcvhiVj27W6xlVxLnv316hiFc9bWJa9NKydRcahK4m7tUM6jNDd8sPCzsLE/NLMwtDw0tzi9tr+40fdf1BTE4DTzaajXVanKpuX9vdO/aq5/7kYtw2duDo9tV2w5AUclcApBuWw8IDYrLu5v7m7nR9dTo5NjU4vz0/OREJQgU2/GaxGWOTTy99yCdXbj6rXf+6LN/fDaWbn1yYaFkdKs2I4Iw3/aEr1qBSh1jbWH5wfZa/qO5namBkZF8Mj2WLrSDwGCO2tUdr90zzFq1lMvcvvPeP33vh98uYKBEi4P3EyvZlqn7hMQGAZ+ojkuJt7edntzazt3c2rs3fu96dnNrplB2mGVzZliu4TWzbb/RrOUz6fnT56tx1ACcXpiv1LuJZkszvUAElHueYVFKAm9qevL+zNrBx9mp9cWBj1LbGzcnNruuH/jc0HVbsfNd1TDbWmM7e32GHMGpd3gn02l1DNMkDsAw5lld6pGgvbm+sZot2FOt6c2B6cFuqzT6sKIFrqlZvZbWMdR2Jr2T2tior1/794+jQzjZya5nNtS67oceB1+EUjDDcYRLtNTsJ+nUyMjV9NbanfHlkSvjQ3uFKne9gBLVtYxeq17fSy3na7sXX7twBBd1MyO5UmCZTihESACOUEcPWMjVrY2ZqlqdG9xc3pm8s7SwOn5/db/E/R4JDKvnG418odMu7C+Wkh89/2/HcG56Iw+WJVyEXsghbyTEaAhXteXx4QeJexMXZ6fGb3wwfvPKrUv3FnaEb3maqhum18tVe91OOtUuz125dgxXHtxJKp5qu4wDJCQiBEogiAA5o1BiVpJ4D4gixgMII+Y4luUSQ61n6pXdYrWb2bw3yMQhXHazmalxles1Q0jSbkMdEFQoUgS2YQRR6FuK6zPe4zY1uOdyIRiDHLRcvWqqDzevb9VKk7mJPzv99Tkd4czM6p0742t7GzMrid3U1L2H21szucV5pbicqI8ldxoTmY2Jg0x1ZbqUXcsmW6vrmVVzI5HZ6vCdNVNTdkq9UrGwNXf5s0N6iIGS74AUFtNtx/Q9T3c1rWUY3Y5lWaJSVzoqKbcKRaNayec3kqlOrrOx1CscJDJefWlsfGTo5szK+v0bC3f3/2EuVtZazKtBCLlAQQuKTmDMh3CRAiwHVRVCUAgIR84pxCW896UTCEZcw1C9RrZe6rSa2fTaD/5yV8auGEsbLoVaBzaD0gvRwqlPJcIS+MU5C5gvOHdo4HuMwlVIfdvRTafd6WyuDKRyOyuXr7/9ld94xYpt1+pSND6HCiwEuBZkERLKLgAzFkLxCzjxueVyAoJySGwltV89KOSzvmaWMvv7+vL14TNvPvurr35Bw5xN5g2b+BC3GBaSwjeF8KPEp0IQSh1iQYXzbZMFFg1AQkIgJbu1Tlkr1IuFTCKZ//AnZ/78S5//8bdctF3LgZrOwn7tB4vBAsNJrC3wD993HZBI6IYX2h51fMPzXcVyvWqvXtpbn5h6f/3q2TN/8+IXn/nm79QAzi6oYH+wO+gWgpIQtxRNBwqCMcFynu5AlrpQ8X1oQx5jVqVbqHXqurp6bXBQL6XPnT3z8pee+eU3XmgCHLMwNON0kNB4YPk+eA48DYYDoV0o4+Bg7lMoEBa2EbuiZIutg1YxUUhsF2vVzltvff8zn/mVH36NAZzT0qDdEEj+fsZGERcB2BFaWhwe4F+IEbgDWiVsC/6Hd5QFQaC5ZnbixlJq/KOz3/3a7/3BP7+AjZFrZn4lgpyc0vYTyztTueVsenshlS6A1tgnOecCgw40BxNCN+a225UPb08OfXKZtx7m9nPTs1de+/LnXnjjtzxUVvMEi7iM0omD+UImvbZbyucWVxIFCX0SxA3jHwgdQALjQt9UsrlGMd3KpfazhUqlmNXGLr/+2umX3vttC11RtQN4hMrthY2dkTvbq8ul8Vv/ccGPQoJ/DxkYP0SrcoxpbPwQmXDhuj3baMzn85nF6z/4zumv/snF5wjAUY8ioZByfnj9wcpoMry/VA3GR+dX46ARmB1gL6AbAgHBtCFkIoY4h07QK6vt4uI7r//ni3/7V29/A5XtdD2CISfyuznl7kEy053b7GQrxXZEUao4GGVMWTigIMfpzG6V6t1Os93AtpjLWRM3z3/v+X8Z/muE890AGAykayK3Xp1dKKcPSmN3z3/03nAUYjVFAiUijEu4FvgG+RTcLsD7UWAYjt2cXhn+xjdfOh/DVXI9YgUUnwx5xGJGFsPE+REdsiWkbOCHCL7BgmA7DvWAOL31Ha20OnLlzedfenHoOxgoWjGptxr1WrvdgmYCWrRrhUKtrfR69Vq1XG2rrUqxVGu0e6aitJrNTrvd6SkHLfhTvVopFSCMq39/+vd/7blnX/mlFpaAliGiiu2qnKZbkoJ1mKPZwOSiWMZDUgheZegIJJWgRhLEFB4whY5te/t3P/juF5/5zbe+7sUFyiU8e/f26MfKnVsD7w9c+iR/cezm0q0bG2vnzr/TvXh/bOR204HcgESm6BcsZPsSq6BCem213ExuvvuP//rc777zBQpwimZi0qiT60F7whFw3YNiEtNbica0/LoDdlcFhagGOSHtpEhLAUkHtiu3lVI9e+PSVz7/7JlfBziZTjngOUKLpYNOGnKfuZbrY5rSENMYlCKQX+El691bow8ADsqCEFkJXVK4ZqfbtWy7/PH7f/GHX771VQzjZMnHRsoiRvLnOlQGjkNNHjMCAvsA5QswiDmdVdaWorBPig8i7CXC1zVD1fWdvWv/9fKf3vieQLiqTyHFyVLS5LuWA0WEeBaBmIMHQDUwFJbpkJvT05u7UUzpI5HD6gg7clPzbb1ZHH/rzVdv/R0y94XZmsko8xKJgyDpeBD8UNcxA2K7g+WFDNCSZGDu7gOQLoIdeAGqFwP25ttgGL/uTQ8NvHYd4cKVdMuEoASmzsvvKoxKYVETWpULpR0HGLAbOoEHQ6XhORwhsFkWkMhgMbRsKLS9+sEnI5fvv4LSdfcqFrQE4CB+865GfLmr7xTTasEqW/V6rVNUuxlUOXRuLGYfRjLuJLQQ4hjABbWhjAZOefcnF96deRXgRCrTsR1g7EArG7ege7NktFKaCXacWb6zW95w9g5y2EEEnVtN7EA9QRMEBWQyYAHHdigYUK3sL91fmpMgXXFbVT0POp5FandrYeA/lAFqJ0BHzFsmU1BcZeheH

19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0
19:52:40, INFO Received command c on object id p0

In [0]:
display_images(BinaryToImage().transform(doc_df))

Origin: dbfs:/FileStore/examples/ocr/images/tiff/00556614_00556648.tif 
 Resolution: 72 dpi 
 Width: 784 px 
 Height: 1000 px 
 Mode: Greyscale 
 Number of channels: 1

19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0
19:54:22, INFO Received command c on object id p0

## Define pipeline

In [0]:
from sparkocr.utils import get_vocabulary_dict

binary_to_image = BinaryToImage()\
    .setOutputCol("image") \
    .setImageType(ImageType.TYPE_3BYTE_BGR)

img_to_hocr = ImageToHocr()\
    .setInputCol("image")\
    .setOutputCol("hocr")\
    .setIgnoreResolution(False)\
    .setOcrParams(["preserve_interword_spaces=0"])

doc_class = VisualDocumentClassifierLilt() \
    .pretrained("lilt_rvl_cdip_296K", "en", "clinical/ocr") \
    .setInputCol("hocr") \
    .setOutputCol("label")

# OCR pipeline
pipeline = PipelineModel(stages=[
    binary_to_image,
    img_to_hocr,
    doc_class
])

19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:15, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:16, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0
19:57:17, INFO Received command c on object id p0


## Call pipeline

In [0]:
results = pipeline.transform(doc_df).cache()

19:59:25, INFO Received command c on object id p0
19:59:25, INFO Received command c on object id p0
19:59:25, INFO Received command c on object id p0
19:59:25, INFO Received command c on object id p0

## Show results

In [0]:
results.select(["label"]) \
    .show(50, truncate=False)

+-------+
label |
+-------+
invoice|
+-------+

20:00:32, INFO Received command c on object id p0
20:00:32, INFO Received command c on object id p0
20:00:32, INFO Received command c on object id p0
20:00:32, INFO Received command c on object id p0